<a href="https://colab.research.google.com/github/Talha1818/AWS_Create_Bucket_Upload_Download_Delete_and_Get_all_Files_from_S3_Bucket/blob/master/AWS_Create_Bucket_Upload_Download_Delete_and_Get_all_Files_from_S3_Bucket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.9 MB/s eta 0:00:00


## Create Bucket (Optional)

In [ ]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            # Create an S3 resource
            s3_client = boto3.resource('s3', aws_access_key_id = 'XXX',
                          aws_secret_access_key= 'XXX')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            # s3_client = boto3.client('s3', region_name=region)
            # Create an S3 resource
            s3_client = boto3.resource('s3', aws_access_key_id = 'XXX',
                          aws_secret_access_key= 'XXX', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
create_bucket("dmsai", region=None)

True

## Upload, Download, Delete and Get all Files from S3 Bucket

In [ ]:
import os
import boto3

class S3Uploader:
    def __init__(self, aws_access_key_id, aws_secret_access_key):
        self.s3 = boto3.resource('s3', aws_access_key_id=aws_access_key_id,
                                 aws_secret_access_key=aws_secret_access_key)
        self.s3_client = boto3.client('s3', aws_access_key_id=aws_access_key_id,
                                      aws_secret_access_key=aws_secret_access_key)

    def upload_file(self, bucket_name, file_name="", key_prefix=""):
        key_prefix_with_file_name = os.path.join(key_prefix, file_name)
        self.s3.Bucket(bucket_name).upload_file(file_name, key_prefix_with_file_name)
        print(f"Uploaded to s3://{bucket_name}/{key_prefix_with_file_name}")

    def download_file(self, bucket_name, file_key, local_file_path):
        self.s3.Bucket(bucket_name).download_file(file_key, local_file_path)
        print(f"Downloaded file '{file_key}' to '{local_file_path}'")

    def list_files_in_bucket(self, bucket_name, folder):
        # List objects in the bucket
        response = self.s3_client.list_objects_v2(Bucket=bucket_name, Prefix=folder)

        # Extract file keys from the response
        file_keys = []
        if 'Contents' in response:
            for obj in response['Contents']:
                file_keys.append(obj['Key'])

        return file_keys

    def delete_file(self, bucket_name, object_key):
        self.s3_client.delete_object(Bucket=bucket_name, Key=object_key)
        print(f"Deleted object '{object_key}' from bucket '{bucket_name}'")

In [ ]:
bucket_name='dmsai'
s3Object = S3Uploader(aws_access_key_id='XXX',
                      aws_secret_access_key='XXX')

In [ ]:
s3Object.upload_file(bucket_name=bucket_name, file_name='20220531-Westpac-62b50dcee4b01b296330c0ea.pdf',
                     key_prefix='Documents')
s3Object.upload_file(bucket_name=bucket_name, file_name='20220531-Westpac-62b50dcee4b01b296330c0ea.pdf',
                     key_prefix='Process_Files')

Uploaded to s3://dmsai/Documents/20220531-Westpac-62b50dcee4b01b296330c0ea.pdf
Uploaded to s3://dmsai/Process_Files/20220531-Westpac-62b50dcee4b01b296330c0ea.pdf


In [ ]:
s3Object.download_file(bucket_name=bucket_name, file_key='Process_Files/20220531-Westpac-62b50dcee4b01b296330c0ea.pdf', local_file_path='sample.pdf')

Downloaded file 'Process_Files/20220531-Westpac-62b50dcee4b01b296330c0ea.pdf' to 'sample.pdf'


In [ ]:
pdf_files = s3Object.list_files_in_bucket(bucket_name=bucket_name, folder='Process_Files')[1:]
pdf_files

['Process_Files/20220531-Westpac-62b50dcee4b01b296330c0ea.pdf']

In [ ]:
s3Object.delete_file(bucket_name=bucket_name, object_key='Documents/Document Managed layout..pdf')

Deleted object 'Documents/Document Managed layout..pdf' from bucket 'dmsai'
